In [1]:
# Add project src to path.
import set_path

import numpy as np

from src.utils.clip_utils import cylinder_clip
from src.utils.las_utils import get_bbox_from_las_file, read_las, label_and_save_las

In [7]:
in_file = '../datasets/pointcloud/grown_2397_9705.las'
out_file = '../datasets/pointcloud/cars_2397_9705.laz'

In [3]:
tilecode = get_bbox_from_las_file(in_file)

In [4]:
print(tilecode)

((119300, 485150), (119350, 485100))


In [5]:
center_point = (119320.032,485101.222)

In [8]:
pointcloud = read_las(in_file)
las_labels = pointcloud.label
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T

In [11]:
from src.utils.labels import Labels
import numpy as np

In [17]:
mask = las_labels == Labels.GROUND
len(pointcloud.z[mask])

print(np.average(pointcloud.z[mask]))

0.4967587400971777


In [20]:
exclude_labels = (1,2)

In [21]:
mask = np.full(len(las_labels), True)

for exclude_label in exclude_labels:
    mask = mask & (las_labels != exclude_label)

In [22]:
labeltje = cylinder_clip(points[mask, :], center_point, 0.6)

In [23]:
mask_indices = np.where(mask)[0]
label_mask = np.zeros(len(mask), dtype=bool)

In [24]:
label_mask[mask_indices[labeltje]] = True

In [25]:
las_labels[label_mask] = 5


In [26]:
label_and_save_las(pointcloud, las_labels, out_file)

# Region Growing

Starting from seed points, the region-growing algorithm is used to decide whether a point should be included in a region or not. Two different implementations are explained in this notebook:
- Region growing based on the Point Cloud Library (PCL) implementation
- Clustering based region growing using "Label Connected Components"


In [ ]:
# Add project src to path.
import set_path

import numpy as np
import pylas
import time

from src.utils.las_utils import label_and_save_las
from src.utils.labels import Labels
import src.region_growing as growing
from src.fusion_pipeline import FusionPipeline

In [ ]:
# In this example we want to grow the building points, the initial seed points.
# The ground points must be removed to fast and properly cluster the cloud.
seed_point_label = Labels.BUILDING
exclude_labels = (Labels.GROUND,)

## Region growing segmentation based on PCL

### Algorithm explained
Based on: https://pcl.readthedocs.io/projects/tutorials/en/latest/region_growing_segmentation.html

With the PCL implementation, a point is picked with minimum curvature value and the region growth process is started. In our implementation, we start from initial seed points that are considered to be inside the object to be segmented. 

For every point in the seed list, the algorithm finds neighboring points. Every neighbor is tested for the angle between its normal and normal of the current seed point. If the angle is less than the threshold value then current point is added to the current region.

In [ ]:
# Region growing with building points as initial seed points. 
region_growing_pcl = growing.RegionGrowing(Labels.BUILDING, exclude_labels)

## Clustering based region growing segmentation

### Algorithm explained
This code performs clustering based region growing segmentation. The main tasks include: 
- The [Label Connected Components Segmentation](https://www.cloudcompare.org/doc/wiki/index.php?title=Label_Connected_Components) algorithm clusters the selected cloud(s) in smaller parts separated by a minimum distance.
- Next, our code grows regions, or in this case all points inside a clusters, based on a percentage of initial seed points inside a cluster. 

### Installation instructions
A Python wrapper provided for CloudCompare allows to use “Label Connected Components”. The Python wrapper works on **linux only**. This wrapper and all the dependencies can be installed using the build and installation instructions in the README at [CloudCompare-PythonPlugin.](https://github.com/tmontaigu/CloudCompare-PythonPlugin.git)

In [ ]:
# Region growing with building points as initial seed points. 
region_growing_clustering = growing.LabelConnectedComp(Labels.BUILDING, exclude_labels,
                                                       octree_level=10, min_component_size=100)

## Process point clouds and save the results

In [ ]:
# Set-up region growing based on PCL pipeline.
growers = (region_growing_pcl,)
pipeline = FusionPipeline(region_growing=growers)

In [ ]:
# Set-up clustering based region growing pipeline.
growers = (region_growing_clustering,)
pipeline = FusionPipeline(region_growing=growers)

### Process a single file

In [ ]:
# Select the file to process. The outfile can be set to 'None' to overwrite the file.
in_file = '../datasets/pointcloud/labelled_2386_9702.laz'
out_file = '../datasets/pointcloud/grown_2386_9702.laz'

# Process the file.
start = time.time()
pipeline.process_file(in_file, out_file=out_file)
end = time.time()
print(f'Tile labelled in {end-start:.2f} seconds.')

### Process a folder

In [ ]:
# Select the folder to process. 
in_folder = '../datasets/pointcloud/'
# Output folder. 'None' uses the input folder.
out_folder = None
# Suffix to add to the filename of processed files. An empty string indicates 
# that the same filename is kept; when out_folder=None this means overwriting.
suffix = '_grown'

# Process the folder.
pipeline.process_folder(in_folder, out_folder=out_folder, suffix=suffix)